# From the github repo

In [76]:
import pickle
import numpy as np
from matplotlib import pyplot as plt
from time import sleep

# Linien imports
from linien_client.device import Device
from linien_client.connection import LinienClient
from linien_common.common import AutolockMode

#import matplotlib
#matplotlib.use("TkAgg")  # or "Qt5Agg"

# ----------------------------------------------
# Connection setup
# ----------------------------------------------
dev = Device(
    host="192.168.1.159",  # IP address of your Red Pitaya
    username="root",
    password="root"
)

c = LinienClient(dev)
c.connect(autostart_server=True, use_parameter_cache=True)

# Choose autolock algorithm:
#   AutolockMode.SIMPLE  -> CPU-based, faster but less robust
#   AutolockMode.ROBUST  -> FPGA-based, slower startup but stable
c.parameters.autolock_mode_preference.value = AutolockMode.ROBUST
sleep(1)  # optional short delay to ensure the setting is applied

# ----------------------------------------------
# Helper: wait until laser is locked/unlocked
# ----------------------------------------------
def wait_for_lock_status(should_be_locked):
    """Wait until the laser reaches the desired lock state."""
    counter = 0
    while True:
        to_plot = pickle.loads(c.parameters.to_plot.value)
        is_locked = "error_signal" in to_plot

        if is_locked == should_be_locked:
            break

        counter += 1
        if counter > 10:
            raise Exception("waited too long")

        sleep(1)


# ----------------------------------------------
# Main autolock procedure
# ----------------------------------------------

# Turn off the lock (if currently running)
c.connection.root.start_sweep()


linien_client.connection       INFO     Try to connect to 192.168.1.159:18862
linien_client.connection       DEBUG    Sent authentication hash
linien_client.connection       INFO     Connection established!


In [77]:

# Wait until the laser is unlocked
wait_for_lock_status(False)

# Record a reference spectrum
to_plot = pickle.loads(c.parameters.to_plot.value)
error_signal = to_plot["error_signal_1"]

# Plot the reference and monitor signals with two y-axes
plt.plot(error_signal)
plt.show()
plt.plot(to_plot["monitor_signal"])
plt.show()

# Ask user for target line region
print("Please specify the position of the target line.")
x0 = int(input("Enter index of a point on the LEFT side of the target line: "))
x1 = int(input("Enter index of a point on the RIGHT side of the target line: "))

# Visualize selected lock region
plt.axvline(x0, color="r")
plt.axvline(x1, color="r")
plt.plot(error_signal)
plt.title("Selected lock region")
plt.show()

# Start autolock with selected region

signal_len = len(error_signal)
print(f"Selected region: {x0}–{x1} of {signal_len} points")

if not (0 <= x0 < x1 <= signal_len):
    raise ValueError("Invalid region: indices outside range or reversed")

if x1 - x0 < 10:
    raise ValueError("Selected region too narrow — pick a wider range")

#sleep(2)

c.connection.root.start_autolock(x0, x1, pickle.dumps(error_signal))

# Wait until laser is actually locked
try:
    wait_for_lock_status(True)
    print("Locking the laser worked! \\o/")
except Exception:
    print("Locking the laser failed :(")
    c.connection.root.start_sweep()


Please specify the position of the target line.
Selected region: 1700–1770 of 2048 points
Locking the laser failed :(


In [62]:
c.connection.root.start_sweep()

In [46]:
counter = 0
limit = 3

while True:

    if counter >= limit:
        break

    counter +=1

    # Fetch the most recent data again after locking
    plot_data = pickle.loads(c.parameters.to_plot.value)

    plt.clf()

    if c.parameters.lock.value:
        plt.figure(figsize=(10, 6))
        plt.title("Laser locked — check signals")
        plt.plot(plot_data["control_signal"], label="Control signal")
        plt.plot(plot_data["error_signal"], label="Error signal")

    # Optional: plot the monitor signal if available
    if "monitor_signal" in plot_data:
        plt.plot(plot_data["monitor_signal"], label="Monitor signal", linestyle="--")

    else:
        plt.figure(figsize=(10, 6))
        plt.title("Laser sweeping/unlocked")
        plt.plot(plot_data["error_signal_1"], label="Error signal ch1")
        plt.plot(plot_data["error_signal_2"], label="Error signal ch2")

        # Again, include monitor signal if present
        if "monitor_signal" in plot_data:
            plt.plot(plot_data["monitor_signal"], label="Monitor signal", linestyle="--")

    plt.legend()
    plt.xlabel('Sample index')
    plt.ylabel('Amplitude (arb. units)')
    plt.grid(True)
    plt.show()

    sleep(1)